任务: 基于flare文本数据，建立LSTM模型，预测序列文字:
1.完成数据预处理，将文字序列数据转化为可用于LSTM输入的数据
2.查看文字数据预处理后的数据结构，并进行数据分离操作
3.针对字符电输入(”flare is a teacher in ai industry，He obtained his phd in Australia.")，预测其对应的后续
字符
备注:模型结构:单层LSTM，输出有20个神经元: 每次使用前20个字符预测第21个字符

In [5]:
# load the data
data = open('flare.txt').read()
# 移除换行符
data = data.replace('\n','').replace('\r','')
print(data)

Certainly, here's a short story about the workplace:Alison had always been ambitious. As a child, she had dreamed of working in a high-powered job in the city, and had worked tirelessly to achieve that goal. Now, at thirty-two, she was a successful executive at one of the biggest companies in the country. But as she sat in her office, staring at the papers on her desk, she couldn't help but feel unfulfilled.Alison had always thought that success would bring her happiness. But as she climbed the corporate ladder, she had become increasingly disillusioned. The long hours, the endless meetings, the constant pressure to perform - it all seemed like a never-ending cycle of stress and anxiety. She longed for something more, but she didn't know what.One day, as she was walking to the office, she noticed a small bookstore nestled in between the buildings. Curiosity got the better of her, and she decided to pop in. As she browsed the shelves, she stumbled upon a book called "Find Your Passion: 

In [7]:
# 字符去重处理
letters = list(set(data))
print(letters)
num_letters = len(letters)
print(num_letters)

['S', 'G', 's', 'B', ' ', 'y', 'C', '"', ',', 'n', 'c', 'x', 'F', 'g', 'o', 'w', ':', 'i', 'e', 'v', 'k', 'f', 'l', 'd', 'P', 'r', 'b', 'Y', 'T', 'W', 't', 'h', 'a', 'A', 'I', '.', 'D', 'L', 'O', 'N', '-', 'm', 'j', 'z', 'p', "'", 'q', 'u']
48


In [9]:
# 建立字典
# int to char
int_to_char = {a:b for a,b in enumerate(letters)}
print(int_to_char)
# char to int
char_to_int = { b:a for a,b in enumerate(letters)}
print(char_to_int)

{0: 'S', 1: 'G', 2: 's', 3: 'B', 4: ' ', 5: 'y', 6: 'C', 7: '"', 8: ',', 9: 'n', 10: 'c', 11: 'x', 12: 'F', 13: 'g', 14: 'o', 15: 'w', 16: ':', 17: 'i', 18: 'e', 19: 'v', 20: 'k', 21: 'f', 22: 'l', 23: 'd', 24: 'P', 25: 'r', 26: 'b', 27: 'Y', 28: 'T', 29: 'W', 30: 't', 31: 'h', 32: 'a', 33: 'A', 34: 'I', 35: '.', 36: 'D', 37: 'L', 38: 'O', 39: 'N', 40: '-', 41: 'm', 42: 'j', 43: 'z', 44: 'p', 45: "'", 46: 'q', 47: 'u'}
{'S': 0, 'G': 1, 's': 2, 'B': 3, ' ': 4, 'y': 5, 'C': 6, '"': 7, ',': 8, 'n': 9, 'c': 10, 'x': 11, 'F': 12, 'g': 13, 'o': 14, 'w': 15, ':': 16, 'i': 17, 'e': 18, 'v': 19, 'k': 20, 'f': 21, 'l': 22, 'd': 23, 'P': 24, 'r': 25, 'b': 26, 'Y': 27, 'T': 28, 'W': 29, 't': 30, 'h': 31, 'a': 32, 'A': 33, 'I': 34, '.': 35, 'D': 36, 'L': 37, 'O': 38, 'N': 39, '-': 40, 'm': 41, 'j': 42, 'z': 43, 'p': 44, "'": 45, 'q': 46, 'u': 47}


In [11]:
# time_step
time_step = 20

In [10]:
#批量字符数据预处理
import numpy as np
from keras.utils import to_categorical
#滑动窗口提取数据
def extract_data(data,slide):
    x = []
    y = []
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])
    return x,y
#字符到数字的批量转换
def char_to_int_Data(x,y,char_to_int):
    x_to_int = []  
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])
    return x_to_int,y_to_int

#实现输入字符文章的批量处理,输入整个字符,滑动窗口大小,转化字典
def data_preprocessing(data,slide,num_letters,char_to_int):
    char_data = extract_data(data,slide)
    int_data = char_to_int_Data(char_data[0],char_data[1],char_to_int)
    Input = int_data[0]
    Output = list(np.array(int_data[1]).flatten()  )
    Input_RESHAPED = np.array(Input).reshape(len(Input ),slide)
    new = np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new,Output

In [12]:
# extract X and y from text data
X,y = data_preprocessing(data,time_step,num_letters,char_to_int)

In [14]:
print(X.shape)

(2058, 20, 48)


In [16]:
print(len(y))

2058


In [17]:
# split the data
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=10)
print(X_train.shape,len(y_train))

(1852, 20, 48) 1852


In [18]:
y_train_category = to_categorical(y_train,num_letters)
print(y_train_category)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
# set up the model
from keras.models import Sequential
from keras.layers import Dense,LSTM

model = Sequential()
# input_shape 看样本的
model.add(LSTM(units=20,input_shape=(X_train.shape[1],X_train.shape[2]),activation="relu"))
#输出层 看样本有多少页
model.add(Dense(units=num_letters ,activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20)                5520      
                                                                 
 dense (Dense)               (None, 48)                1008      
                                                                 
Total params: 6528 (25.50 KB)
Trainable params: 6528 (25.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# train the model
model.fit(X_train,y_train_category,batch_size=100,epochs=200)

Epoch 1/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0414 - accuracy: 0.6825
Epoch 2/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0697 - accuracy: 0.6674
Epoch 3/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0269 - accuracy: 0.6836
Epoch 4/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0211 - accuracy: 0.6884
Epoch 5/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0192 - accuracy: 0.6803
Epoch 6/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0209 - accuracy: 0.6841
Epoch 7/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0077 - accuracy: 0.7025
Epoch 8/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0217 - accuracy: 0.6917
Epoch 9/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0668 - accuracy: 0.6609
Epoch 10/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0735 - accuracy: 0.6685
Epoch 11/

In [24]:
# make prediction based on the training data
y_train_predict = np.argmax(model.predict(X_train),1)
print(y_train_predict)

58/58 [==============================] - 0s 1ms/step
[ 9 25 18 ... 18 17  4]


In [25]:
# trainsform the int to letters
y_train_predict_char = [int_to_char[i] for i in y_train_predict]
print(y_train_predict_char)

['n', 'r', 'e', ' ', ' ', ' ', 'r', ' ', 'o', 's', 'o', ' ', 'e', 'e', 'h', 'i', 'd', 'l', ' ', 'l', 't', 't', 'l', 't', 'h', 'e', 'n', 't', 'a', 'l', 'A', ' ', ',', 't', 't', 'e', 't', 'I', 't', ' ', 'o', 'n', ':', 'r', 't', 'h', ' ', 'n', 'u', 'i', 'n', 'v', 's', 's', 's', 'i', 'f', 't', 'h', 'e', 'e', 'l', 'h', 't', ' ', 'e', 'e', 't', 'i', ' ', 'h', 'e', 'd', 'h', 't', 'c', 'u', ' ', 'r', 'n', 'o', ' ', ' ', 's', ' ', 'a', ' ', 'r', ' ', ' ', 'h', 'j', 'i', ' ', 'b', 'i', 'n', ' ', ' ', 'r', ' ', 'r', ' ', 'n', 'r', 'n', ' ', 'n', 'i', 'd', 'Y', 't', ' ', 's', 's', 'e', 'a', 't', 'k', 'd', ' ', 'd', 'e', 'e', 'd', 'o', 's', 'r', 'h', 'h', 'e', ' ', ' ', 't', 'm', 'l', 'y', 'o', 'e', 't', 'd', 'h', 'r', 't', 't', 'r', 'l', 't', 't', ' ', 's', 'i', 'r', 'e', 'a', 's', 'n', 'n', 'n', 'h', 'n', 'o', ' ', 't', 'e', 'e', 't', 'e', 's', 'i', 's', 'n', 't', 'o', ' ', ' ', 't', 'e', 't', ' ', 'e', 'i', 'A', 'o', 'o', 'v', ' ', 's', 'b', 'l', 'd', 'o', 't', 'u', 's', 's', 't', 'h', 'r', 't',

In [26]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train,y_train_predict)
print(accuracy)

0.6830453563714903


In [27]:
y_test_predict = np.argmax(model.predict(X_test),1)
accuracy_test = accuracy_score(y_test,y_test_predict)
print(accuracy_test)
print(y_test_predict)
print(y_test)

7/7 [==============================] - 0s 2ms/step
0.3446601941747573
[18 17 30  4 10 18 23 14  4 23 18 18 18  4  4 14 18 14 31  9 30 30 15 30
 31 15  4 26 23  4 18  2  2 35 31 18 23  9  9 33 25 18 13 18 26 23 30 22
 17 18  4 23  4  4 18 15  9  2 18  2 30  4 30 22 23 47  9 14 30  4 18 32
 30 14  4 30 22  4 22 10  2 14  2  8  4  2 13 31  4  2  9 32  9  4 23  9
  4  9  9 22 14 31 31 22 18 13  4 18 35 30  4  4 18 23  9 20 47 31  9  4
 18 18 23  4 22  2 22 15 14 18 25 18 23  8 22 14 18 31  4 13  4  2  4 23
  4 23 31 18 18 25 30 30 33 18  4  4 32 23 18 17 25 30  4  4 32  4  9 18
 31 17 18  8  9  4 18 31 14  9  2 18  8 14  2 14 14 15 14  9  4 21 38 18
 14 30 18  4 30 14 18 20 25  4  4 18 18  2]
[18, 4, 26, 8, 26, 18, 45, 10, 4, 25, 31, 18, 22, 4, 4, 32, 32, 14, 32, 4, 14, 21, 31, 17, 18, 26, 4, 4, 32, 18, 17, 2, 5, 40, 31, 18, 23, 22, 11, 6, 47, 14, 44, 26, 2, 14, 4, 30, 18, 30, 4, 23, 4, 4, 18, 30, 19, 2, 18, 32, 44, 4, 14, 15, 32, 31, 9, 31, 4, 4, 4, 18, 2, 21, 4, 4, 9, 4, 14, 10, 30, 31, 

In [35]:
new_letters = 'Alison had always thought that success would bring her happiness'
X_new, y_new = data_preprocessing(new_letters,time_step,num_letters,char_to_int)
y_new_predict = np.argmax(model.predict(X_new),1)
print(y_new_predict)

2/2 [==============================] - 0s 2ms/step
[17 47 13 31 30  4 30 31 32 30  4  2 31 41 10 17  2  2  4 10 32 25 22 23
  4 32 18 18  9 13  4 30 18 25  4 30 18 22 30 17  9 13  2  8]


In [36]:
y_new_predict_char = [int_to_char[i] for i in y_new_predict ]
print(y_new_predict_char)

['i', 'u', 'g', 'h', 't', ' ', 't', 'h', 'a', 't', ' ', 's', 'h', 'm', 'c', 'i', 's', 's', ' ', 'c', 'a', 'r', 'l', 'd', ' ', 'a', 'e', 'e', 'n', 'g', ' ', 't', 'e', 'r', ' ', 't', 'e', 'l', 't', 'i', 'n', 'g', 's', ',']


In [37]:
for i in range(0,X_new.shape[0]-20):
    print(new_letters[i:i+20],'--predict next letter is --',y_new_predict_char[i])

Alison had always th --predict next letter is -- i
lison had always tho --predict next letter is -- u
ison had always thou --predict next letter is -- g
son had always thoug --predict next letter is -- h
on had always though --predict next letter is -- t
n had always thought --predict next letter is --  
 had always thought  --predict next letter is -- t
had always thought t --predict next letter is -- h
ad always thought th --predict next letter is -- a
d always thought tha --predict next letter is -- t
 always thought that --predict next letter is --  
always thought that  --predict next letter is -- s
lways thought that s --predict next letter is -- h
ways thought that su --predict next letter is -- m
ays thought that suc --predict next letter is -- c
ys thought that succ --predict next letter is -- i
s thought that succe --predict next letter is -- s
 thought that succes --predict next letter is -- s
thought that success --predict next letter is --  
hought that success  --predict 

LSTM文本生成实战summary:
1、通过搭建LSTM模型，实现了基于文本序列的字符生成功能
2、学习了文本加载、字典生成方法;
3、掌握文本的数据预处理方法，并熟悉了转化数据的结构:
4、实现了对新文本数据的字符预测